# This is code creates the emaildb database from flat file

In [1]:
"""
Created on Wed Jul 28 08:07:56 2021

@author: cbaku

Class Examples: Relational Databases using SQLite

"""

import sqlite3

sql_handle = sqlite3.connect('emaildb.sqlite')    #create the file when it runs
sql_crsr = sql_handle.cursor()

sql_crsr.execute('''
                 DROP TABLE IF EXISTS Counts''')
                 
sql_crsr.execute('''
                 CREATE TABLE Counts (email TEXT, count INTEGER)''')

f_name = input('Enter file name:')
if (len(f_name) < 1):
    f_name = 'mbox-short.txt'

f_handle = open(f_name)
for line in f_handle:
    if not line.startswith('From: '):
        continue
    words = line.split()
    email_address = words[1]
    sql_crsr.execute('''SELECT count FROM Counts WHERE email = ?''', \
                     (email_address,))
    # DO NOT USE string addition 'SELECT ... + email_address' for constructing
    # this sql statement as it can enable SQL injection
    #
    # (email_address,) is a one member tuple used as a parameter
    # The cursor helps to get to the specific record with in the database
    sql_row = sql_crsr.fetchone()
    if sql_row is None:
        sql_crsr.execute('''INSERT INTO Counts (email, count) \
                 VALUES (?,1)''', (email_address,))
    else:
        sql_crsr.execute('''UPDATE Counts SET count = count +1 \
                         WHERE email = ?''', (email_address,))
                         
    # updating a value using a SQL statement is better than fetching
    # the value, incrementing it in Python and them updating the new
    # value in SQL. The reason is other applications may be trying to update
    # the same table at that time. UPDATEing at the database level will
    # keep the process in sync with other processes. It will be ATOMic.
    
    sql_handle.commit()

# print the contents of the count statement
#
#sql_str = '''SELECT email, count FROM Counts \
#                ORDER BY email \
#                LIMIT 20 '''

sql_str = '''SELECT email, count FROM Counts \
                ORDER BY count DESC \
                LIMIT 20 '''

for row in sql_crsr.execute(sql_str):
    print(str(row[0]),row[1])
    
sql_crsr.close()
            

Enter file name:


FileNotFoundError: [Errno 2] No such file or directory: 'mbox-short.txt'